In [130]:
import numpy as np ## load in numpy
import pandas as pd ## loading in panda commands

In [131]:
## loading in and setting variables preliminary manipulation ## easy to read but complex version
caulf = pd.read_csv('C:/Users/Julius/Downloads/caulfielddistrict.csv') ## caulfield 2018 state booth data
sandy = pd.read_csv('C:/Users/Julius/Downloads/sandringhamdistrict.csv')
bright = pd.read_csv('C:/Users/Julius/Downloads/brightondistrict.csv')
bent = pd.read_csv('C:/Users/Julius/Downloads/bentleighdistrict.csv')


In [132]:
## Turning the raw state data into a workable formatw
ALP_2pp = list(bent['ALP_Approx_%']) + list(bright['ALP_Approx_%']) + list(caulf['ALP_Approx_%']) + list(sandy['ALP_Approx_%']) ## make electorate 2pp into list
Booth = list(bent['Polling_booth']) + list(bright['Polling_booth']) + list(caulf['Polling_booth']) + list(sandy['Polling_booth']) ## make booth of 2pp into list

## Turning these lists into one data frame
State_results = pd.DataFrame(
    {'2pp': ALP_2pp,
     'Booth': Booth
    })

## removing the useless data points
Not_wanted = ('Marked as Voted','Early Vote','Postal Vote','Provisional','TOTAL ALL VOTE TYPES',
              'Total Ordinary Votes','Absent','Percentage of Formal Vote\nPolled By Candidate\n') 
State_results_final = State_results[~(State_results['Booth'].isin(Not_wanted)) & (State_results['2pp'] != 'NaN')]
State_booths = list(State_results_final['Booth'])

In [133]:
## Loading in the deferal data
All = pd.read_csv('C:/Users/Julius/Downloads/HouseTppByPollingPlaceDownload-24310.csv')
Gold = All[All['DivisionNm'] == 'Goldstein']

In [135]:
## Isolating which booths have direct replacements available, and which will need a sub
Not_in_State = Gold[~(Gold['PollingPlace'].isin(State_booths))]
In_state = Gold[(Gold['PollingPlace'].isin(State_booths))]
Covered = list(In_state['PollingPlace'])

In [137]:
## Create dictionary with fed terms to be replaced and state equiv
d = {'Bentleigh Central (Goldstein)':'Bentleigh Central','Brighton South':'Dendy','Beaumaris East (Goldstein)':'Mentone','Moorabbin South (Goldstein)':'Moorabbin South',
    'Elsternwick North (Goldstein)':'Elsternwick North','Cheltenham (Goldstein)':'Cheltenham','Highett (Goldstein)':'Highett South','Highett West':'Highett',
    'Glenhuntly':'Ormond','Glenhuntly East (Goldstein)':'Glenhuntly East','McKinnon East (Goldstein)':'McKinnon East','Elsternwick (Goldstein)':'Elsternwick',
    'Elsternwick Central':'Clarence'}

In [147]:
## Invert disctionary into a useful tool for the state data points into federal ones
inv_d = {v: k for k, v in d.items()}

# Setting Goldstein at start
Gold = All[All['DivisionNm'] == 'Goldstein']

## Drop the votes columns
del Gold['Liberal/National Coalition Votes']
del Gold['Liberal/National Coalition Percentage']
del Gold['Australian Labor Party Votes']
del Gold['TotalVotes']
del Gold['Swing']

## Create a list of booths + ALP % for ones without state equivalent
Gold_non_reps = Gold.replace(d) ## create Goldstein results with state booth names
Gold_non_replacements = Gold_non_reps[~(Gold_non_reps['PollingPlace'].isin(State_booths))] ## remove state booth names to leave fed only

## Setup for next step, create list of replaceable Goldstein polling places, get all data from those places
lst_Gold_non_reps = list(Gold_non_reps['PollingPlace'])
Gold_reps = Gold_non_reps[(Gold_non_reps['PollingPlace'].isin(State_booths))]

## Append Federal data to have state TPP in place of fed
State_results_final.replace(d) # Change state booths to new names
State_results_replacements = State_results_final[State_results_final['Booth'].isin(lst_Gold_non_reps)] # Filter non Goldstein polling booth names

## create polling places list
pp = list(State_results_replacements['Booth'])
pp2 = list(Gold_non_replacements['PollingPlace'])

## rejoining of data sets
Gold_reps.set_index('PollingPlace', inplace = True) # Index Gold with replacement by polling place, 
State_results_replacements.set_index('Booth', inplace = True) # Index state level with replacement by polling place,
State_results_replacements['PollingPlace2'] = pp # add back in polling place column
intr = Gold_reps.join(State_results_replacements) # join df's, 
del intr['Australian Labor Party Percentage'] # drop Australian Labor Party Percentage, 
ALP = intr['2pp']*100 # make percentages from 0.x
del intr['2pp'] # delete state 2pp
intr['Australian Labor Party Percentage'] = ALP # add in state 2pp
Gold_non_replacements.set_index('PollingPlace', inplace = True) # set index to booth
Gold_non_replacements['PollingPlace2'] = pp2 # add back in polling place
Final = Gold_non_replacements.merge(intr, how= 'outer') # merge dataframes

## renaming variables
pp = Final['PollingPlace2'] # make list of polling place2
del Final['PollingPlace2']
Final['PollingPlace'] = pp # add in polling place again
Final = Final.replace(inv_d)
## print out results to CSV
Final.to_csv('C:/Users/Julius/Downloads/Fed2019_via_State.csv', sep=',')

<ipython-input-147-86006a9a9fa1>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  State_results_replacements['PollingPlace2'] = pp # add back in polling place column
<ipython-input-147-86006a9a9fa1>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Gold_non_replacements['PollingPlace2'] = pp2 # add back in polling place
